# Introduction

## Stock Price Prediction

The core purpose of this project is to predict the future price of a given stock. We would try to predict if a stock price would go up or down based on previous stock price. A thorough technical analysis of a stock is very crucial for traders to open a position on that stock and optimize their profit.

In the following notebook, data wrangling part has been done on stock price data where we have prepared the data for further exploratory data analysis. 


## Table of Contents:
* [Import Libraries](#import-libraries)
* [Sector to ticker mapping](#split-into-sectors)
* [Cleaning the data](#cleaning-the-data)
* [Filtering the data](#filtering-the-data)
* [Downsampling the data](#downsampling-the-data)
* [Pulling revenue data](#pulling-revenue-data)
* [Pulling interest rate data](#pulling-interest-rate-data)
* [Merging the data](#merging-the-data)
* [Pickling the dictionary](#pickling-the-dictionary)



## Importing the necessary libraries <a class="anchor" id="import-libraries"></a>


In [1551]:
import pandas as pd
import pandas
import datetime as dt
from datetime import datetime
from urllib.error import HTTPError
import pandas_market_calendars as mcal
import requests
from bs4 import BeautifulSoup
import re
import csv
import numpy as np
from sklearn import datasets, linear_model, metrics
import pickle

## Sector to ticker mapping <a class="anchor" id="split-into-sectors"></a>

For each sector, creating two dictionaries: one with ticker as key and stock timeseries dataframe as value, other one with ticker as key and company name as value. 

Sector to ticker mapping data has been obtained using web scraping from https://www.stockmonitor.com/sectors/

In [375]:
def create_dfs(ticker_csv):
    df_dict = {}
    ticker_cmpny = {}
    null_vals = []
    ticker = pd.read_csv(ticker_csv) 
    for ticker,company_name in zip(ticker['Ticker'],ticker['Company_Name']):
        filename = "Data/Stocks/" + ticker.lower() + ".us.txt"
        ticker_name = ticker
        compny_name = company_name
        try:
            compny_df = pd.read_csv(filename)
            compny_df.set_index('Date',inplace = True)
            compny_df.index = pd.to_datetime(compny_df.index)
            compny_df.drop(columns = 'OpenInt', axis = 1, inplace = True)
            compny_df['Daily_Returns'] = compny_df['Close'].pct_change()
            compny_df.drop(index = compny_df.index[0], axis = 0, inplace = True )
            for val in compny_df.isna().sum():
                if val != 0:
                    null_vals.append(ticker_name)
            df_dict[ticker_name] = compny_df
            ticker_cmpny[ticker_name] = compny_name
        except FileNotFoundError:
            pass
        except EmptyDataError:
            print ("file " + filename + " is empty !!!")
    print (null_vals)        
    return (df_dict,ticker_cmpny)

"""    
communication_sector_dfs, = create_dfs("communication_service_ticker.csv")
materials_sector_dfs = create_dfs("materials_ticker.csv")
consumer_cyclical_dfs = create_dfs("consumer_cyclical_ticker.csv")
consumer_defensive_dfs = create_dfs("consumer_defensive_ticker.csv")
energy_sector_dfs = create_dfs("energy_ticker.csv")
financial_service_dfs = create_dfs("financial_ticker.csv")
healthcare_sector_dfs = create_dfs("healthcare_ticker.csv")
industrial_sector_dfs=create_dfs("industrial_ticker.csv")
technology_sector_dfs=create_dfs("technology_ticker.csv")
utility_sector_dfs=create_dfs("utility_ticker.csv")
"""

consumer_cyclical_dfs, ticker_cmpny_dict = create_dfs("consumer_cyclical_ticker.csv")

print(consumer_cyclical_dfs, ticker_cmpny_dict)




[]
{'AAN':               Open    High     Low   Close   Volume  Daily_Returns
Date                                                              
2005-02-28  13.371  13.470  13.118  13.381   683911       0.006469
2005-03-01  13.371  13.794  13.342  13.734   624585       0.026381
2005-03-02  13.607  14.038  13.568  13.822   387289       0.006407
2005-03-03  13.891  13.891  13.450  13.627   362061      -0.014108
2005-03-04  13.685  13.968  13.549  13.832   311148       0.015044
...            ...     ...     ...     ...      ...            ...
2017-11-06  36.280  36.570  35.570  35.590  1029506      -0.023058
2017-11-07  35.720  35.720  34.290  34.440  1149793      -0.032312
2017-11-08  35.510  36.090  34.800  35.990  1805497       0.045006
2017-11-09  35.680  35.860  35.230  35.620   979951      -0.010281
2017-11-10  37.600  38.060  36.260  36.590  1214968       0.027232

[3200 rows x 6 columns], 'ANF':               Open    High      Low   Close   Volume  Daily_Returns
Date             

## Cleaning the data by imputing missing days <a class="anchor" id="cleaning-the-data"></a>

Cleaning the data by finding out the missing days in stock market time series. We have compared the the time series dates with New York Stock Exchange (NYSE) open dates. If any company has more than passed in missing days limit, we have dropped the company for further analysis, otherwise we have merged the time series dataframe with the NYSE data frame using right join and finally impute the missing days with 'ffill' method. 



In [376]:
def extract_missing_days(df1, df2, missing_days_dict):
            missing_days_list = []
            for missing_days in df1.index.difference(df2.index):
                    #print (missing_days)
                    missing_days_list.append(missing_days)
            missing_days_dict[keys] = missing_days_list
            #print (missing_days_dict)

def probe_missing_data(df, missing_days_limit, missing_days_dict=None):
    list_missing_data_gt_limit = []
    nyse = mcal.get_calendar('NYSE')
    for keys in df:
        start_date = pd.to_datetime(df[keys].index.min())
        end_date = pd.to_datetime(df[keys].index.max())
        
        #create a df for nyse open days with date as index
        nyse_open_days = nyse.schedule(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'))
        date_index = nyse_open_days['market_open'].index 
        df_nyse = pd.DataFrame()
        df_nyse.index = date_index
        
        #checking if nyse open days equal to the stock price time series 
        if len(nyse_open_days) == df[keys].shape[0]:
            continue
        else:
            if missing_days_dict:
                extract_missing_days(df_nyse, df[keys], missing_days_dict)
            if len(nyse_open_days) - df[keys].shape[0] >= missing_days_limit:
                list_missing_data_gt_limit.append(keys)
            else:
                merged_df = df[keys].merge(df_nyse, how = 'right', right_index = True, left_index = True)
                merged_df.fillna(method = 'ffill',inplace = True)
                #print (merged_df.isna().sum())
                df[keys] = merged_df
                #print (len(nyse_open_days),df[keys].shape[0] )
    
    #print(len(list_missing_data))
    for ele in list_missing_data_gt_limit:
        del df[ele]
    return df

cons_cyc_dfs_cor_days = probe_missing_data(consumer_cyclical_dfs, 100)
     
cons_cyc_dfs_cor_days
    

{'AAN':               Open    High     Low   Close     Volume  Daily_Returns
 2005-02-28  13.371  13.470  13.118  13.381   683911.0       0.006469
 2005-03-01  13.371  13.794  13.342  13.734   624585.0       0.026381
 2005-03-02  13.607  14.038  13.568  13.822   387289.0       0.006407
 2005-03-03  13.891  13.891  13.450  13.627   362061.0      -0.014108
 2005-03-04  13.685  13.968  13.549  13.832   311148.0       0.015044
 ...            ...     ...     ...     ...        ...            ...
 2017-11-06  36.280  36.570  35.570  35.590  1029506.0      -0.023058
 2017-11-07  35.720  35.720  34.290  34.440  1149793.0      -0.032312
 2017-11-08  35.510  36.090  34.800  35.990  1805497.0       0.045006
 2017-11-09  35.680  35.860  35.230  35.620   979951.0      -0.010281
 2017-11-10  37.600  38.060  36.260  36.590  1214968.0       0.027232
 
 [3201 rows x 6 columns],
 'ANF':               Open    High      Low   Close     Volume  Daily_Returns
 2005-02-28  44.722  44.936  43.8890  44.274  1

## Filtering data by years <a class="anchor" id="filtering-the-data"></a>

Filtering data by years, deleting 'OpenInt' column since it has no information and adding a new column that computes daily returns

In [1558]:

def filter_data_by_year(df,start_year,end_year):
    new_df = pd.DataFrame()
    df_start_year = df.index.year.min()
    df_end_year = df.index.year.max()
    if start_year >= df_start_year:
        if end_year <= df_end_year:
            new_df = df.loc[pd.to_datetime(str(start_year)):pd.to_datetime(str(end_year + 1))]
    return (new_df)
            
def filter_data_by_year_dict_list(df_dict,start_year,end_year):
    new_df = {}
    for key in df_dict:
        filtered_df = filter_data_by_year(df_dict[key],start_year,end_year)
        if not filtered_df.empty:
            new_df[key] = filtered_df
    return (new_df)


cons_cyc_df_years = filter_data_by_year_dict_list(cons_cyc_dfs_cor_days, 2006,2017)






## Downsampling the data from daily to quarterly <a class="anchor" id="downsampling-the-data"></a>

In [1546]:

cons_cyc_df_q_resample = {}


for keys in cons_cyc_df_years: 
    try:
        cons_cyc_df_q_resample[keys] = cons_cyc_df_years[keys].resample('Q').mean()
    except TypeError as e:
        print ("Caught exception while running {}: {}".format(keys, str(e)))



## Pulling the revenue data <a class="anchor" id="pulling-revenue-data"></a>

Pulling revenue data for each company from https://www.macrotrends.net/stocks/ using web scraping 
and creating a dictionary with ticker name as key and revenue data as value.  

In [454]:
def fetchRevenueData(ticker_csv, start_year, end_year):
    
    stock_df, cmpny_name = create_dfs(ticker_csv)
    #print (cmpny_name)
    stock_df = probe_missing_data(stock_df, 100)
    stock_df = filter_data_by_year_dict_list(stock_df,start_year,end_year)
    ticker_revenue_dicts = {}
    p_e_ratio_dicts = {}
    for keys in stock_df: 
        ticker_name = keys
        company_name = cmpny_name[keys]
        
        #print (ticker_name,company_name)
        res = re.split(' |, ', company_name)
        macrotrends_name = ''
        for elem in res:
            if not macrotrends_name:
                macrotrends_name = elem.lower()
            else:
                macrotrends_name = macrotrends_name + "-" + elem.lower()
        #test if the ticker name combo works on Macrotrends
                macrotrends_rev_url = "https://www.macrotrends.net/stocks/charts/{}/{}/revenue".format(ticker_name, macrotrends_name)
                macrotrends_pe_url = "https://www.macrotrends.net/stocks/charts/{}/{}/pe-ratio".format(ticker_name, macrotrends_name)
                x = requests.head(macrotrends_rev_url)
                if (x.status_code != 404):
                    # fetch revenue data using read_html()
                    try:
                        revenue = pd.read_html(macrotrends_rev_url , match = 'Quarterly Revenue', flavor = 'bs4')[0]
                        pe_ratio = pd.read_html(macrotrends_pe_url, match = 'PE Ratio Historical Data', flavor = 'bs4')[0]
                        ticker_revenue_dicts[ticker_name] = revenue
                        p_e_ratio_dicts[ticker_name] = pe_ratio
                        #print (p_e_ratio_dicts)
                        #with open('ticker_revenue.csv', 'w') as f:
                             #f.write("%s,%s\n"%(key,ticker_revenue_dicts[key]))  
                    except HTTPError as e:
                        print ("Error while retriving URL \"" + macrotrends_rev_url + "\": " + str(e))
                        print ("Error while retriving URL \"" + macrotrends_pe_url + "\": " + str(e))
                        pass
        
    #print (ticker_revenue_dicts)
    #clean and filter the revenue data
    cmpny_revenue = {}
    for keys in ticker_revenue_dicts:
        revenue_df = ticker_revenue_dicts[keys].set_index(ticker_revenue_dicts[keys].columns[0]).rename_axis('')
        revenue_df = revenue_df.rename(columns = {revenue_df.columns[0] :'Quarterly Revenue(Millions of US $)' })
        revenue_df.index = pd.to_datetime(revenue_df.index) 
        revenue_df = revenue_df.sort_index()
        
        revenue_df = filter_data_by_year(revenue_df,start_year,end_year)
        if revenue_df.empty:
            continue
        revenue_df['Quarterly Revenue(Millions of US $)'] = revenue_df['Quarterly Revenue(Millions of US $)'].str.replace('$','')
        
    #clean and filter the p/e ratio data
            
        pe_df = p_e_ratio_dicts[keys].set_index(p_e_ratio_dicts[keys].columns[0]).rename_axis('')
        pe_df.drop([pe_df.columns[0],pe_df.columns[1]], axis = 1, inplace = True)
        pe_df.columns = pe_df.columns.droplevel()
        pe_df.index = pd.to_datetime(pe_df.index)
        pe_df = pe_df.sort_index()
        pe_df = filter_data_by_year(pe_df,start_year,end_year)
        if pe_df.empty:
            continue
        rev_pe = revenue_df.merge(pe_df, left_index = True, right_index = True)
        cmpny_revenue[keys] = rev_pe
        
    return (cmpny_revenue)
    
                                                                              
                                                                              
#new_revenue = fetchRevenueData("consumer_cyclical_ticker.csv", 2009, 2017)
new_revenue_2006_2017 = fetchRevenueData("consumer_cyclical_ticker.csv", 2006, 2017)
new_revenue_2007_2017 = fetchRevenueData("consumer_cyclical_ticker.csv", 2007, 2017)

new_revenue_2007_2017




[]
Error while retriving URL "https://www.macrotrends.net/stocks/charts/GES/guess'-inc/revenue": HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently
Error while retriving URL "https://www.macrotrends.net/stocks/charts/GES/guess'-inc/pe-ratio": HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently


/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_19153/687011807.py:52: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  revenue_df['Quarterly Revenue(Millions of US $)'] = revenue_df['Quarterly Revenue(Millions of US $)'].str.replace('$','')


[]
Error while retriving URL "https://www.macrotrends.net/stocks/charts/GES/guess'-inc/revenue": HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently
Error while retriving URL "https://www.macrotrends.net/stocks/charts/GES/guess'-inc/pe-ratio": HTTP Error 301: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Moved Permanently


/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_19153/687011807.py:52: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  revenue_df['Quarterly Revenue(Millions of US $)'] = revenue_df['Quarterly Revenue(Millions of US $)'].str.replace('$','')


{'ANF':            Quarterly Revenue(Millions of US $)  PE Ratio
                                                         
 2007-01-31                               1,139     11.91
 2007-04-30                                 742     12.17
 2007-07-31                                 805     10.09
 2007-10-31                                 974     11.05
 2008-01-31                               1,179     10.57
 2008-04-30                                 800      9.81
 2008-07-31                                 846      7.32
 2008-10-31                                 896      4.32
 2009-01-31                                 942      4.12
 2009-04-30                                 602      9.34
 2009-07-31                                 637     22.98
 2009-10-31                                 754     38.69
 2010-01-31                                 936     63.29
 2010-04-30                                 688     58.78
 2010-07-31                                 746     25.43
 2010-1

## Pulling the interest rate data <a class="anchor" id="pulling-interest-rate-data"></a>

Pulling interest rate data from 'fed-funds-rate-historical-chart.csv', creating a dataframe
with interest rate as column, filtering the data by years and finally resample it on quarterly basis 

In [501]:
#fetch and clean interest rate data

interest_rates = pd.read_csv('fed-funds-rate-historical-chart.csv')
interest_rates.set_index('date',inplace = True)
interest_rates = interest_rates.rename_axis('')
interest_rates.index = pd.to_datetime(interest_rates.index)
#interset_rates = interset_rates.rename(columns = {'value' : 'Interest Rate'}, inplace = True)
interest_rates.columns = ['Interest Rate (%)']
interest_rates = interest_rates.loc[pd.to_datetime('2006'):pd.to_datetime('2018')].resample('Q').mean()


## Merging the data <a class="anchor" id="merging-the-data"></a>

Merge the revenue data and interset rate with the close price from stock dataframe and put it as
value of dictionary where key is company name

In [984]:
new_revenue_2006_2017
exo_df = {}

for keys in cons_cyc_df_q_resample:
    try:
        com_df = pd.merge_asof(cons_cyc_df_q_resample[keys]['Close'], new_revenue_2006_2017[keys], left_index = True,
                                              right_index = True)
        com_df = pd.merge_asof(com_df, interest_rates, right_index = True, left_index = True)
        
        exo_df[keys] = com_df
        
        exo_df[keys]['Quarterly Revenue(Millions of US $)'] = exo_df[keys]['Quarterly Revenue(Millions of US $)'].str.replace(',','').astype('float64')
        
        values = {'Quarterly Revenue(Millions of US $)': exo_df[keys]['Quarterly Revenue(Millions of US $)'].fillna(method = 'bfill') 
         , 'PE Ratio': exo_df[keys]['PE Ratio'].median() }
        
        exo_df[keys].fillna(value = values, inplace = True)
        
    except KeyError:
        print (keys)
    
    except AttributeError:
        pass
   
 
exo_df

AAN
TAST
GDEN
GES
MLCO
SBH


{'ANF':                 Close  Quarterly Revenue(Millions of US $)  PE Ratio  \
 2006-03-31  51.854435                                863.0     24.10   
 2006-06-30  48.363508                                863.0     24.10   
 2006-09-30  49.529444                                863.0     24.10   
 2006-12-31  59.525381                                863.0     12.36   
 2007-03-31  64.063934                               1139.0     11.91   
 2007-06-30  65.478651                                742.0     12.17   
 2007-09-30  62.030556                                805.0     10.09   
 2007-12-31  65.511938                                974.0     11.05   
 2008-03-31  62.851607                               1179.0     10.57   
 2008-06-30  59.155172                                800.0      9.81   
 2008-09-30  42.972812                                846.0      7.32   
 2008-12-31  19.707906                                896.0      4.32   
 2009-03-31  17.564131                      

## Pickling the dictionary <a class="anchor" id="pickling-the-dictionary"></a>

In [1559]:
with open('exo_df.pickle', 'wb') as handle:
    pickle.dump(exo_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

with open('cons_cyc_df_years.pickle', 'wb') as handle:
    pickle.dump(cons_cyc_df_years, handle, protocol=pickle.HIGHEST_PROTOCOL)   

# Summary
    
    

In the above notebook, data wrangling part has been done on stock price data. We have divided
the different company based on the sectors they belong. Then we picked one sector and clean the data.
Basically, we have fixed a cut-off and compare our data with NYSE open days. If there is missing days
more than cut-off days, we have dropped the company, otherwise we have imputed using 'ffill' method.
Then we have filtered the data by years. Here we have taken data from 2006 to 2016.The 
frequency of data is daily; we have downsampled the data to quarterly for further analysis.
Using web scraping we have pulled quarterly revenue data for each of the company. Also, we have 
made a dataframe containing the interest rate data from 2006 to 2016. Finally we have merged the
revenue and interest rate data frame to the 'close' column from stock price data. So, our final 
datatype is dictionary where company ticker name is key and value is dataframe with columns including
close, revenue, PE ratio and interest rate.

Now, in the next notebook we are planning to do exploratory analysis with our current data. 
